## Loading the dataset
in this we dont have sentiment column so we make that with the help of review
  

In [1]:
import pandas as pd

df = pd.read_csv("G:\\Step_internship\\STEP_INTERNSHIP\\all_kindle_review.csv")

df['Sentiment'] = df['rating'].apply(lambda x: 1 if x > 3 else (0 if x < 3 else None))

df.dropna(subset=['Sentiment'], inplace=True)

print(df.head())

   Unnamed: 0.1  Unnamed: 0        asin helpful  rating  \
1             1        5957  B002HJV4DE  [1, 1]       5   
4             4        1776  B001A06VJ8  [0, 1]       4   
5             5        3744  B0021L9YDK  [6, 6]       5   
6             6       13641  B0038NN38W  [1, 1]       2   
7             7        4448  B002AJ7X2C  [1, 1]       4   

                                          reviewText   reviewTime  \
1  Great short read.  I didn't want to put it dow...   10 8, 2013   
4  I did not expect this type of book to be in li...  12 31, 2012   
5  Aislinn is a little girl with big dreams. Afte...   12 7, 2009   
6  This has the makings of a good story... unfort...  08 18, 2011   
7  I got this because I like collaborated short s...   03 8, 2010   

       reviewerID                                      reviewerName  \
1  A2RGNZ0TRF578I                                      Holly Butler   
4  A3C9V987IQHOQD                                          Rjostler   
5  A3J5NN6MJK4M4A

## preprocessing the data
convertiing to lower case and removing unnessary puntuation and special charactr etc

In [2]:
import re
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

df['reviewText'] = df['reviewText'].apply(preprocess_text)

print(df.head())


[nltk_data] Downloading package stopwords to C:\Users\M.Vishnu
[nltk_data]     Balaji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   Unnamed: 0.1  Unnamed: 0        asin helpful  rating  \
1             1        5957  B002HJV4DE  [1, 1]       5   
4             4        1776  B001A06VJ8  [0, 1]       4   
5             5        3744  B0021L9YDK  [6, 6]       5   
6             6       13641  B0038NN38W  [1, 1]       2   
7             7        4448  B002AJ7X2C  [1, 1]       4   

                                          reviewText   reviewTime  \
1  great short read didnt want put read one sitti...   10 8, 2013   
4  expect type book library pleased find price right  12 31, 2012   
5  aislinn little girl big dreams death older bro...   12 7, 2009   
6  makings good story unfortunately disappointsit...  08 18, 2011   
7  got like collaborated short stories alot times...   03 8, 2010   

       reviewerID                                      reviewerName  \
1  A2RGNZ0TRF578I                                      Holly Butler   
4  A3C9V987IQHOQD                                          Rjostler   
5  A3J5NN6MJK4M4A

## TD-IDF


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['reviewText'])
y = df['Sentiment']


## Word2vec

In [6]:
from gensim.models import Word2Vec
import numpy as np

df['reviewText_Tokens'] = df['reviewText'].apply(lambda x: x.split())


word2vec_model = Word2Vec(sentences=df['reviewText_Tokens'], vector_size=100, window=5, min_count=1, workers=4)

def get_word2vec_vector(text):
    vector = [word2vec_model.wv[word] for word in text if word in word2vec_model.wv]
    return sum(vector) / len(vector) if vector else np.zeros(100)

X_word2vec = np.array(df['reviewText_Tokens'].apply(get_word2vec_vector).tolist())
y = df['Sentiment']


## Training the classifier
    1.Split the dataset into train and test sets
    2.Train a Logistic Regression classifier
    3.Predict on the test set
    4.# Evaluate the classifier

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Train a Logistic Regression classifier
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.88


## Full code with TD-IFD

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Train a Logistic Regression classifier
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 0.88


## Full code with Word2Vec

In [9]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the dataset
df = pd.read_csv("G:\\step_intern\\all_kindle_review.csv")

# Add the Sentiment column based on the rating
# Ratings 4 and 5 are considered positive (1), 1 and 2 are negative (0)
df['Sentiment'] = df['rating'].apply(lambda x: 1 if x > 3 else (0 if x < 3 else None))

# Drop neutral reviews (rating = 3)
df.dropna(subset=['Sentiment'], inplace=True)

# Preprocess text
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

df['reviewText'] = df['reviewText'].apply(preprocess_text)

# Tokenize the review text
df['reviewText_Tokens'] = df['reviewText'].apply(lambda x: x.split())

# Train a Word2Vec model
word2vec_model = Word2Vec(sentences=df['reviewText_Tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Convert review text to Word2Vec vectors
def get_word2vec_vector(text):
    vector = [word2vec_model.wv[word] for word in text if word in word2vec_model.wv]
    return sum(vector) / len(vector) if vector else np.zeros(100)

X_word2vec = np.array(df['reviewText_Tokens'].apply(get_word2vec_vector).tolist())
y = df['Sentiment']

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)

# Train a Logistic Regression classifier
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


[nltk_data] Downloading package stopwords to C:\Users\M.Vishnu
[nltk_data]     Balaji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.78
